In [2]:
import os
import sys
import json
import albumentations as A

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback
from building_footprint_segmentation.trainer import Trainer

### Initalize type of segmentation to run for building footprint

In [3]:
segmentation = init_segmentation("binary")

### Defining augmenters to use

In [12]:
augmenters = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2)
]).to_dict()

In [13]:
transfer_weights="/Users/dorianaunito/PostDoc/PROJECTS/SOGEI-DevOps/venv/model/refine.pth"

### Load essential segmentation objects for training

In [11]:
model = segmentation.load_model(name="ReFineNet", transfer_weights=transfer_weights, pre_trained_image_net=False,
        top_layers_trainable=False)
criterion = segmentation.load_criterion(name="BinaryCrossEntropy")
loader = segmentation.load_loader(
    root_folder=r"/Users/dorianaunito/PostDoc/PROJECTS/SOGEI-DevOps/venv/building-footprint-segmentation/data",
    image_normalization="divide_by_255",
    label_normalization="binary_label",
    augmenters=augmenters,
    batch_size=2,
)
metrics = segmentation.load_metrics(
    data_metrics=["accuracy", "precision", "f1", "recall", "iou"]
)

optimizer = segmentation.load_optimizer(model, name="Adam")

callbacks = CallbackList()
# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in  ["TrainChkCallback", "TimeCallback", "TensorBoardCallback", "TrainStateCallback"]:
    callbacks.append(load_callback(r"/Users/dorianaunito/PostDoc/PROJECTS/SOGEI-DevOps/venv/building-footprint-segmentation/out_data", caller))

TypeError: list indices must be integers or slices, not str

### Initalize the Trainer object


In [6]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)

### Start training


In [7]:
trainer.train(start_epoch=0, end_epoch=1)

Epoch 0, lr 0.001:   0%|                                                                                                                                                                      | 0/8 [00:00<?, ?it/s]C:\Users\Fuzail\anaconda3\envs\building-footprint-segmentation\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Epoch 0, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:12<00:00,  9.04s/it, loss=2.67997]


>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 2.6799654960632324, validation: 0.6257065534591675
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 1:Minutes 34:Seconds

### Resuming Training


In [8]:
path_to_saved_state = r"D:\Cypherics\lib_check\out_data\state\default.pt"

# if new_end_epoch is None, the trainer will resume and run till end_epoch, the value of which is saved in state file 
trainer.resume(state=path_to_saved_state, new_end_epoch=2)

Epoch 1, lr 0.001:   0%|                                                                                                                                                                      | 0/8 [00:00<?, ?it/s]C:\Users\Fuzail\anaconda3\envs\building-footprint-segmentation\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Epoch 1, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:05<00:00,  8.20s/it, loss=0.64895]


>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.6489536613225937, validation: 0.6272180676460266
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 1:Minutes 16:Seconds